In [ ]:
import bpy
import mathutils
import numpy as np

In [ ]:
import geomlib_blender as gb

In [ ]:
# create a 3d point cloud derived from the covariance matrix

mesh = bpy.data.meshes.new(name="pointcloud")
obj = bpy.data.objects.new("pointcloud_obj", mesh)
bpy.context.collection.objects.link(obj)
bpy.context.view_layer.objects.active = obj
# bpy.ops.object.mode_set(mode='OBJECT')

points = np.random.multivariate_normal(np.zeros(3),cov,(1_000_000,))
mesh.from_pydata(points,[],[])

In [ ]:
A = np.random.normal(0,10,(3,3))
cov = np.matmul(A.T,A)
np.sqrt(np.linalg.eig(cov)[0])

In [ ]:
# 3d ellipsoid
val,vec = np.linalg.eig(cov)

mat = np.eye(4)
mat[:3,:3] = vec*np.sqrt(val[np.newaxis,:])
mat = mathutils.Matrix(mat)

bpy.ops.mesh.primitive_ico_sphere_add(
    subdivisions=4,
    radius=1,
    enter_editmode=False,
    align="WORLD",
    location=(0,0,0),
    # scale=np.sqrt(val),
    scale = (1,1,1),
)
bpy.context.object.matrix_world = mat

## Marginal

In [ ]:
cov_inv = np.linalg.inv(cov)
cov_sl = np.linalg.inv(cov_inv[:2,:2])

val_marg,vec_marg = np.linalg.eig(cov[:2,:2])

mat_marg = np.eye(4)
mat_marg[:2,:2] = vec_marg*np.sqrt(val_marg[np.newaxis,:])
mat_marg = mathutils.Matrix(mat_marg)

bpy.ops.mesh.primitive_circle_add(
    enter_editmode=True,
    align="WORLD",
    location=(0,0,0),
    scale=(1,1,1),
)
bpy.ops.mesh.edge_face_add()
bpy.ops.object.editmode_toggle()
bpy.context.object.matrix_world = mat_marg

### conditioned on z=const

$$\begin{bmatrix}x_A\\x_B\end{bmatrix}
\sim
\mathcal{N}
\left(
\begin{bmatrix}\mu_A\\\mu_B\end{bmatrix},
\begin{bmatrix}\Sigma_{AA}&\Sigma_{AB}\\\Sigma_{BA}&\Sigma_{BB}\end{bmatrix}
\right)$$

$$x_{A}|x_{B}\sim\mathcal{N}\left(\mu_{A}+\Sigma_{AB}\Sigma_{BB}^{-1}(x_B-\mu_B),\Sigma_{AA}-\Sigma_{AB}\Sigma_{BB}^{-1}\Sigma_{BA}\right)$$

$$x_{B}|x_{A}\sim\mathcal{N}\left(\mu_{B}+\Sigma_{BA}\Sigma_{AA}^{-1}(x_A-\mu_A),\Sigma_{BB}-\Sigma_{BA}\Sigma_{AA}^{-1}\Sigma_{AB}\right)$$

In [ ]:
# conditional
z = 0

sigma_aa = cov[:2,:2]
sigma_ab = cov[:2,2]
sigma_ba = cov[2,:2]
sigma_bb = cov[2,2]

mu_a = sigma_ab*z/sigma_bb

cov_aa = sigma_aa - np.outer(sigma_ab,sigma_ba)/sigma_bb

val_sl,vec_sl = np.linalg.eig(cov_aa)

mat_sl = np.eye(4)
mat_sl[:2,:2] = vec_sl*np.sqrt(val_sl[np.newaxis,:])
mat_sl[0:3,3] = np.concatenate([mu_a,[z]])
# print(mat_sl)
mat = mathutils.Matrix(mat_sl)
# print(mat_sl)

bpy.ops.mesh.primitive_circle_add(
    enter_editmode=True,
    align="WORLD",
    location=(0,0,0),
    scale=(1,1,1),
)
bpy.ops.mesh.edge_face_add()
bpy.ops.object.editmode_toggle()
bpy.context.object.matrix_world = mat

In [ ]:
# create a 3d point cloud derived from the covariance matrix

mesh = bpy.data.meshes.new(name="pointcloud")
obj = bpy.data.objects.new("pointcloud_obj", mesh)
bpy.context.collection.objects.link(obj)
bpy.context.view_layer.objects.active = obj
# bpy.ops.object.mode_set(mode='OBJECT')

points = np.random.multivariate_normal(np.zeros(3),cov,(1_000_000,))
mesh.from_pydata(points,[],[])